In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd

In [2]:
train_set = pd.read_csv('train.csv', header=None)
test_set = pd.read_csv('test.csv', header=None)

In [3]:
X_train = train_set.iloc[:4000, 1:].to_numpy() # the first 4000 samples as training set
y_train = train_set.iloc[:4000, 0].to_numpy() # first column is the class label
y_train[y_train==0] = -1 # Change 0 to -1
print(X_train.shape)
print(y_train.shape)
X_validate = train_set.iloc[4000:, 1:].to_numpy()
y_validate = train_set.iloc[4000:, 0].to_numpy()
y_validate[y_validate==0] = -1
print(X_validate.shape)
print(y_validate.shape)
X_test = test_set.iloc[:, 1:].to_numpy()
y_test = test_set.iloc[:, 0].to_numpy()
y_test[y_test==0] = -1
print(X_test.shape)
print(y_test.shape)

(4000, 200)
(4000,)
(4500, 200)
(4500,)
(1500, 200)
(1500,)


In [4]:
def svm_train_primal(data_train, label_train, regularisation_para_C):
    m, n = data_train.shape
    
    W = cp.Variable(n) 
    b = cp.Variable()    
    Psi = cp.Variable(m) 
    
    C = regularisation_para_C
    
    objective = cp.Minimize(0.5 * cp.norm(W)**2 + C/m * cp.sum(Psi))
    
    constraints = [cp.multiply(label_train, (data_train @ W + b)) - 1 + Psi >= 0, Psi >= 0]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()
    
    return W.value, b.value, Psi.value

def svm_predict_primal(data_test, label_test, svm_model):
    W, b, _ = svm_model 
    result = data_test @ W + b
    result = np.sign(result)
    accuracy = np.mean(result == label_test)
    return accuracy

primal_model = svm_train_primal(X_train, y_train, 100)
W_primal, b_primal, Psi_primal = primal_model
test_accuracy = svm_predict_primal(X_test, y_test, primal_model)
print("Solution of b:", b_primal)
print("Sum of all dimensions of W solution:", np.sum(W_primal))
print("test accuracy:", test_accuracy)

Solution of b: 1.7798137266344365
Sum of all dimensions of W solution: -0.14521544474819548
test accuracy: 0.968


In [5]:
def svm_train_dual(data_train, label_train, regularisation_para_C):
    N = len(data_train)
    alpha = cp.Variable(N)

    quadratic_term = cp.sum_squares(data_train.T@cp.multiply(alpha, label_train))

    objective = cp.Maximize(cp.sum(alpha) - 0.5 * quadratic_term)
    
    constraints = [
        0 <= alpha,                
        alpha <= regularisation_para_C/N, 
        cp.sum(cp.multiply(alpha, label_train)) == 0 
    ]
    
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.CVXOPT)
    
    alpha_values = alpha.value
    
    return alpha_values

dual_model = svm_train_dual (X_train, y_train, 100)
print("Sum of all dimensions of alpha solution:", np.sum(dual_model))

Sum of all dimensions of alpha solution: 7.281635914644138


In [6]:
def primal_solution_from_dual(data_train, label_train, alpha_values, regularisation_para_C):  
  N = len(data_train)

  w = np.sum(alpha_values[:, None] * label_train[:, None] * data_train, axis=0)

  # Find support vectors (where 0 < alpha_i <= C)
  sv_indices = np.where((alpha_values > 0) & (alpha_values <= regularisation_para_C/N))

  # b* = y_i - w^T * x_i 
  b = np.mean(label_train[sv_indices] - np.dot(data_train[sv_indices], w))

  return w, b

primal_from_dual_model = primal_solution_from_dual(X_train, y_train, dual_model, 100)
W_dual, b_dual = primal_from_dual_model
print("Solution of b:", b_dual)
print("Sum of all dimensions of W solution:", np.sum(W_dual))

Solution of b: 1.740571633690616
Sum of all dimensions of W solution: -0.14520561643482588


In [7]:
def find_support_vectors_from_primal(data_train, label_train, svm_model):
    W, b, Psi = svm_model

    decision_margin = label_train * (data_train @ W + b)
    margin_threshold = 1e-5
    #soft margin, so must remember to include slack variable, previously obtained 180 because only calculated hard margin
    support_vector_indices = np.where((Psi > margin_threshold) | (np.abs(decision_margin - (1 - Psi)) < margin_threshold))
    return data_train[support_vector_indices]

vectors_primal = find_support_vectors_from_primal(X_train, y_train, primal_model)
print("Support vectors from primal:", len(vectors_primal))
vectors_primal

Support vectors from primal: 392


array([[-0.36, -0.91, -0.99, ...,  0.3 ,  2.44, -1.26],
       [ 1.05, -1.79,  0.9 , ...,  0.39,  0.6 , -1.66],
       [ 1.01, -1.13,  1.49, ...,  0.23, -0.3 , -0.01],
       ...,
       [ 2.16, -0.78, -0.78, ..., -0.38,  1.1 ,  0.39],
       [ 0.36, -0.19, -1.06, ..., -0.83, -0.2 ,  0.12],
       [-0.73, -1.19, -0.24, ...,  1.46, -1.36,  1.21]])

In [8]:
def find_support_vectors_from_dual(data_train, alpha_values, margin_threshold=1e-5):
    support_vector_indices = np.where(alpha_values > margin_threshold)[0]
    return data_train[support_vector_indices]

vectors_dual = find_support_vectors_from_dual(X_train, dual_model)
print("Support vectors from dual:", len(vectors_dual))
vectors_dual

Support vectors from dual: 392


array([[-0.36, -0.91, -0.99, ...,  0.3 ,  2.44, -1.26],
       [ 1.05, -1.79,  0.9 , ...,  0.39,  0.6 , -1.66],
       [ 1.01, -1.13,  1.49, ...,  0.23, -0.3 , -0.01],
       ...,
       [ 2.16, -0.78, -0.78, ..., -0.38,  1.1 ,  0.39],
       [ 0.36, -0.19, -1.06, ..., -0.83, -0.2 ,  0.12],
       [-0.73, -1.19, -0.24, ...,  1.46, -1.36,  1.21]])

In [17]:
C_ranges = [2**i for i in range (-10, 11, 2)]
best_C = None
best_accuracy = -1

for C in C_ranges:
    model = svm_train_primal(X_train, y_train, C)
    current_accuracy = svm_predict_primal(X_validate, y_validate, model)
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_C = C
    print(f"C: {C}, Accuracy: {current_accuracy:.4f}")

print ('\nValidation accuracy from optimal C:',best_accuracy)
print ('Optimal C:', best_C)

model = svm_train_primal(X_train, y_train, best_C)
test_accuracy = svm_predict_primal(X_test, y_test, model)
print('\nTest Accuracy using optimal C:', test_accuracy)

C: 0.0009765625, Accuracy: 0.4909
C: 0.00390625, Accuracy: 0.4909
C: 0.015625, Accuracy: 0.4909
C: 0.0625, Accuracy: 0.9244
C: 0.25, Accuracy: 0.9622
C: 1, Accuracy: 0.9718
C: 4, Accuracy: 0.9749
C: 16, Accuracy: 0.9740
C: 64, Accuracy: 0.9713
C: 256, Accuracy: 0.9660
C: 1024, Accuracy: 0.9627

Validation accuracy from optimal C: 0.9748888888888889
Optimal C: 4

Test Accuracy using optimal C: 0.9746666666666667


In [10]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

svm = LinearSVC(C=best_C) #optimal C from previous question
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Test accuracy using optimal C from previous question:", accuracy)
print ('Optimal C:', best_C)

Test accuracy using optimal C from previous question: 0.9713333333333334
Optimal C: 4


In [16]:
#search best C using sklearn
C_ranges = [2**i for i in range(-10, 11, 2)]
best_C = None
best_accuracy = -1

for C in C_ranges:
    svm = LinearSVC(C=C)
    svm.fit(X_train, y_train)
    y_pred_val = svm.predict(X_validate)
    current_accuracy = accuracy_score(y_validate, y_pred_val)
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_C = C
    print(f"C: {C}, Accuracy: {current_accuracy:.4f}")

print('\nValidation accuracy using sklearn:', best_accuracy)
print('Optimal C using sklearn:', best_C)

svm = LinearSVC(C=best_C)
svm.fit(X_train, y_train)
y_pred_test = svm.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test) 
print('\nTest Accuracy using sklearn:', test_accuracy)

C: 0.0009765625, Accuracy: 0.9644
C: 0.00390625, Accuracy: 0.9667
C: 0.015625, Accuracy: 0.9660
C: 0.0625, Accuracy: 0.9653
C: 0.25, Accuracy: 0.9649
C: 1, Accuracy: 0.9640
C: 4, Accuracy: 0.9640
C: 16, Accuracy: 0.9638
C: 64, Accuracy: 0.9636
C: 256, Accuracy: 0.9633
C: 1024, Accuracy: 0.9633

Validation accuracy using sklearn: 0.9666666666666667
Optimal C using sklearn: 0.00390625

Test Accuracy using sklearn: 0.968
